In [23]:
from typing import Any, Dict

import numpy as np
import scipy.sparse as sp
import torch

from src.graph_models.csbm import CSBM

In [24]:
def configure_hardware(device, seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Hardware
    #torch.backends.cuda.matmul.allow_tf32 = other_params["allow_tf32"]
    #torch.backends.cudnn.allow_tf32 = other_params["allow_tf32"]
    if not torch.cuda.is_available():
        assert device == "cpu", "CUDA is not availble, set device to 'cpu'"
    elif device == "cpu":
        device = torch.device("cpu")
    else:
        device = torch.device(f"cuda:{device}")

    return device



In [27]:
# Data
classes = 2
n = 10
#n_per_class_trn = 400
K = 1.5
sigma = 1
avg_within_class_degree = 1.58 * 2
avg_between_class_degree = 0.37 * 2
seed = 42
# Model
depth = 1
dtype = torch.float32

In [28]:
csbm = CSBM(n, avg_within_class_degree, avg_between_class_degree, K, sigma, classes=classes)
X, A, y = csbm.sample(n, seed)
# Row normalize
degree_l = np.sum(A, axis=1, dtype=np.float32)
degree_inv_l = np.power(degree_l, -1)
S = degree_inv_l[:,None] * A
device = configure_hardware(0, seed)
X = torch.tensor(X, dtype=dtype, device=device)
S = torch.tensor(S, dtype=dtype, device=device)
y = torch.tensor(y, device=device)

In [38]:
# Computing NTK
csigma = 1 # ReLU
S_norm = torch.norm(S)
XXT = X.matmul(X.T)
Sig = S.matmul(XXT.matmul(S.T))
kernel = torch.zeros((S.shape), dtype=dtype).to(device)
p = torch.zeros((S.shape), dtype=dtype).to(device)

In [32]:
Diag_Sig = torch.diagonal(Sig)

In [37]:
Diag_Sig.reshape(1, -1).shape

torch.Size([1, 10])

In [ ]:
sig_i = p + 

In [36]:
Diag_Sig.reshape(-1, 1)

tensor([[4.3995],
        [0.9830],
        [1.9854],
        [3.4234],
        [2.4339],
        [0.0201],
        [2.4660],
        [0.8422],
        [1.5574],
        [0.4527]], device='cuda:0')

In [33]:
Diag_Sig

tensor([4.3995, 0.9830, 1.9854, 3.4234, 2.4339, 0.0201, 2.4660, 0.8422, 1.5574,
        0.4527], device='cuda:0')